In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.catboost import CatBoost as Model

In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
model_predictions = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], features)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    m.score(thresh=10)
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    display(m.top_n_predictions(20, worst=True))
    m.calculate_effectiveness()
    global model_predictions
    model_predictions = pd.concat([model_predictions, m.predictions])

In [5]:
feature_sets = dict(
    DEF = ['age', 'season', 'window', 'fee', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo', 'marketval', 
           'matchesplayed', 'minsplayed', 'foot', 'height', 'weight', 'avgpasslen', 'dribbles90', 'acclongpassesct', 'accprpassespct'],
    
    MID = ['age', 'season', 'window', 'fee', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo', 'marketval', 
           'matchesplayed', 'minsplayed', 'foot', 'height', 'weight', 'assists90', 'acccrossesleftpct', 'succssdribblespct'],
    
    ATT = ['age', 'season', 'window', 'fee', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo', 'marketval', 
           'matchesplayed', 'minsplayed', 'foot', 'height', 'weight', 'goals90', 'npgoals90', 'xg90', 'hdrgoals90', 'shots90', 
           'xa90', 'shotassist90', 'touchesbox90', 'prgruns90', 'acclr90', 'rcvdpasses90', 'smartpasses90', 'dribbles90', 
           'succssdribblespct']
)

In [6]:
for pos, features in feature_sets.items():
    pipeline(pos, features, full_feature_set=True)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,minsplayed,foot,height,weight,avgpasslen,dribbles90,acclongpassesct,accprpassespct
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,3128,1,189,89,21.02,0.78,57.58,80.16
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,1175,1,184,79,19.57,1.53,49.09,79.12
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,2096,1,182,74,19.02,6.83,62.50,78.23
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,3356,1,183,72,17.70,4.40,44.00,66.67
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,2809,1,186,78,22.59,1.51,52.28,77.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,2782,1,182,73,21.07,1.20,41.24,62.88
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,860,1,170,60,14.22,6.38,66.67,72.50
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,2959,1,191,84,21.25,1.06,57.46,62.63
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,1421,1,181,78,22.80,0.76,52.38,69.15


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'avgpasslen',
 'dribbles90',
 'acclongpassesct',
 'accprpassespct']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'iterations': 3829, 'learning_rate': 0.04266002496614089, 'depth': 6, 'subsample': 0.2958287486170316, 'colsample_bylevel': 0.70052477226584, 'min_data_in_leaf': 94}
thresh = 10
mae: 2.1703251357853452
mape: 0.4710347845661659
mae_below_thresh: 1.2112228722302083
mape_above_thresh: 0.2544196682231628


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
197,Christian Luyindama,25,19/20,Belgium,BE1,Standard Liège,Türkiye,TR1,Galatasaray,1,7.50,5.500,5.00,5.500,5.493758,0.006242,0.001135
2096,Portu,31,23/24,Spain,ES1,Real Sociedad,Spain,ES1,Getafe CF,1,3.50,3.500,3.50,3.500,3.509292,0.009292,0.002655
989,Wendell,28,21/22,Germany,L1,Bayer 04 Leverkusen,Portugal,PO1,FC Porto,1,6.50,5.000,4.30,5.000,5.020230,0.020230,0.004046
1550,Michal Sadílek,23,22/23,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Twente Enschede FC,1,4.00,4.000,1.50,4.000,4.024155,0.024155,0.006039
21,Konstantinos Manolas,28,19/20,Italy,IT1,AS Roma,Italy,IT1,SSC Napoli,1,45.00,32.000,36.00,32.000,32.025526,0.025526,0.000798
841,Ahmet Oğuz,27,20/21,Türkiye,TR1,Kasimpasa,Türkiye,TR1,Sivasspor,0,0.45,0.800,0.00,0.800,0.829129,0.029129,0.036412
1656,Federico Ricca,27,22/23,Belgium,BE1,Club Brugge KV,Belgium,BE1,Oud-Heverlee Leuven,1,2.00,1.500,0.00,1.500,1.444435,0.055565,0.037044
288,Sergey Parshivlyuk,30,19/20,Russia,RU1,FK Rostov,Russia,RU1,Dinamo Moscow,1,3.00,1.800,0.00,1.800,1.725771,0.074229,0.041239
260,Neven Subotic,30,19/20,France,FR1,AS Saint-Étienne,Germany,L1,1.FC Union Berlin,1,4.00,2.000,0.00,2.000,1.915878,0.084122,0.042061
1250,Gilles Dewaele,25,21/22,Belgium,BE1,KV Kortrijk,Belgium,BE1,Standard Liège,0,1.30,1.400,0.00,1.400,1.512390,0.112390,0.080279


...

['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'goals90',
 'npgoals90',
 'xg90',
 'hdrgoals90',
 'shots90',
 'xa90',
 'shotassist90',
 'touchesbox90',
 'prgruns90',
 'acclr90',
 'rcvdpasses90',
 'smartpasses90',
 'dribbles90',
 'succssdribblespct']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'iterations': 1105, 'learning_rate': 0.0671024742588727, 'depth': 2, 'subsample': 0.35919567461164376, 'colsample_bylevel': 0.1325160127653328, 'min_data_in_leaf': 68}
thresh = 10
mae: 4.972523663130783
mape: 1.1205644496609628
mae_below_thresh: 2.786113857713398
mape_above_thresh: 0.3947116903719235


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
2184,Kike García,33,23/24,Spain,ES1,CA Osasuna,Spain,ES1,Deportivo Alavés,1,1.800,1.50,0.00,1.50,1.436066,0.063934,0.042623
2042,Bertuğ Yıldırım,21,23/24,Türkiye,TR1,Hatayspor,France,FR1,Stade Rennais FC,1,2.700,5.00,5.40,5.00,5.088737,0.088737,0.017747
153,Luciano Vietto,25,19/20,Spain,ES1,Atlético de Madrid,Portugal,PO1,Sporting CP,1,7.000,6.00,11.00,6.00,6.090882,0.090882,0.015147
418,Renaud Emond,28,19/20,Belgium,BE1,Standard Liège,France,FR1,FC Nantes,0,3.500,3.50,4.00,3.50,3.603001,0.103001,0.029429
658,Mustapha Bundu,23,20/21,Denmark,DK1,Aarhus GF,Belgium,BE1,RSC Anderlecht,1,3.000,2.00,2.80,2.00,2.115372,0.115372,0.057686
195,Andy Carroll,30,19/20,England,GB1,West Ham United,England,GB1,Newcastle United,1,8.000,3.20,0.00,3.20,3.077449,0.122551,0.038297
1252,Benjamin Nygren,20,21/22,Belgium,BE1,KRC Genk,Denmark,DK1,FC Nordsjaelland,0,2.000,1.00,0.60,1.00,0.866751,0.133249,0.133249
1584,Alexis Sánchez,33,22/23,Italy,IT1,Inter Milan,France,FR1,Olympique Marseille,1,3.500,4.00,0.00,4.00,4.142529,0.142529,0.035632
2028,Raúl Jiménez,32,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Fulham FC,1,6.000,6.00,6.40,6.00,5.840176,0.159824,0.026637
249,Dmitriy Poloz,27,19/20,Russia,RU1,Zenit St. Petersburg,Russia,RU1,FC Sochi,1,4.000,1.80,3.00,1.80,1.972476,0.172476,0.095820


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1270,Erling Haaland,21,22/23,Germany,L1,Borussia Dortmund,England,GB1,Manchester City,1,150.0,180.0,60.00,180.0,91.634878,88.365122,0.490917
1560,Xavi Simons,19,22/23,France,FR1,Paris Saint-Germain,Netherlands,NL1,PSV Eindhoven,1,4.0,40.0,0.00,40.0,9.221782,30.778218,0.769455
484,André Silva,24,20/21,Italy,IT1,AC Milan,Germany,L1,Eintracht Frankfurt,1,24.0,45.0,3.00,45.0,16.719357,28.280643,0.628459
16,Rafael Leão,20,19/20,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,15.0,20.0,49.50,20.0,42.173678,22.173678,1.108684
1813,Matheus Nunes,25,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Manchester City,1,45.0,55.0,62.00,55.0,76.179161,21.179161,0.385076
495,Alexander Sørloth,24,20/21,England,GB1,Crystal Palace,Germany,L1,RB Leipzig,1,13.0,16.0,20.00,16.0,35.182204,19.182204,1.198888
53,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,25.0,16.0,28.00,16.0,33.549695,17.549695,1.096856
1877,Victor Boniface,22,23/24,Belgium,BE1,Union Saint-Gilloise,Germany,L1,Bayer 04 Leverkusen,1,12.0,40.0,20.50,40.0,23.076671,16.923329,0.423083
918,Luca Waldschmidt,25,21/22,Portugal,PO1,SL Benfica,Germany,L1,VfL Wolfsburg,1,15.0,7.0,12.00,7.0,23.758900,16.758900,2.394129
86,Raúl de Tomás,25,19/20,Portugal,PO1,SL Benfica,Spain,ES1,RCD Espanyol Barcelona,0,20.0,16.0,22.50,16.0,29.747592,13.747592,0.859224


In [7]:
model_predictions

,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,...,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct,age_coef,inflation,eff_true,eff_pred
0,Ismail Jakobs,21,21/22,Germany,L1,1.FC Köln,France,FR1,AS Monaco,1,...,7.0,6.50,7.0,8.266914,1.266914,0.180988,0.451151,0.199843,-0.276282,-0.242255
1,Karlo Bartolec,24,19/20,Denmark,DK1,FC Nordsjaelland,Denmark,DK1,FC Copenhagen,1,...,1.5,2.67,1.5,2.612039,1.112039,0.741359,0.670855,-0.078351,-0.394543,-0.192183
2,Darko Lazović,28,19/20,Italy,IT1,Genoa CFC,Italy,IT1,Hellas Verona,1,...,4.0,0.00,4.0,1.507513,2.492487,0.623122,0.912789,-0.078351,0.660258,0.248837
3,Antonio Barragán,33,20/21,Spain,ES1,Real Betis Balompié,Spain,ES1,Elche CF,1,...,0.8,0.00,0.8,1.584753,0.784753,0.980941,1.397068,-0.233083,0.455417,0.902154
4,Tin Jedvaj,25,21/22,Germany,L1,Bayer 04 Leverkusen,Russia,RU1,Lokomotiv Moscow,1,...,4.0,4.00,4.0,5.877029,1.877029,0.469257,0.741438,0.199843,-0.254703,-0.061385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Raúl Jiménez,32,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Fulham FC,1,...,6.0,6.40,6.0,5.840176,0.159824,0.026637,1.307574,NaN,NaN,NaN
176,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,...,16.0,28.00,16.0,33.549695,17.549695,1.096856,0.604575,-0.078351,-0.350089,-0.119848
177,Michel Vlap,22,19/20,Netherlands,NL1,SC Heerenveen,Belgium,BE1,RSC Anderlecht,1,...,5.5,6.90,5.5,7.024858,1.524858,0.277247,0.531726,-0.078351,-0.232931,-0.177947
178,Pablo Sarabia,27,19/20,Spain,ES1,Sevilla FC,France,FR1,Paris Saint-Germain,1,...,28.0,18.00,28.0,33.244925,5.244925,0.187319,0.866515,-0.078351,0.104063,0.165702


In [8]:
r2 = model_predictions.eff_pred.corr(model_predictions.eff_true)
r2

0.6178834389281407